In [1]:
import pandas as pd
import regex as re
import numpy as np

In [30]:
df = pd.read_csv("~/Desktop/attacks.csv", encoding="latin-1")

In [31]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [32]:
# el nombre de Species tiene un espacio a la derecha
# genericamente quitamos los espacios a los lados
df.columns = df.columns.str.strip()

In [33]:
df.shape

(25723, 24)

In [34]:
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [36]:
# Limpieza básica de strings
for column in df.columns[df.dtypes == object]:
  df[column] = df[column].str.strip() # quitar espacios en blanco a izq y der
  df[column] = df[column].str.lower() # todo a minúsculas
df.replace(r'\s+',' ', inplace = True, regex = True) # espacios multiples a espacio simple


In [37]:
print("Cardinalidad de las variables")
for column in df.columns:
  print(column,"-",df[column].nunique()/df[column].notna().sum())


Cardinalidad de las variables
Case Number - 0.7224775913583085
Date - 0.8606791494763567
Year - 0.039523809523809524
Type - 0.0012702445220704986
Country - 0.032149712092130515
Area - 0.13630921840259963
Location - 0.7023602915654287
Activity - 0.2563390065995137
Name - 0.8562048588312541
Sex - 0.0008715356458079135
Age - 0.042639008931143765
Injury - 0.5782594835830411
Fatal (Y/N) - 0.0008676036786395974
Time - 0.12008141112618724
Species - 0.4102193995381062
Investigator or Source - 0.7820206841686556
pdf - 0.9982545223738496
href formula - 0.9982542453578797
href - 0.9973024436686766
Case Number.1 - 0.9973024436686766
Case Number.2 - 0.9974611234528721
original order - 0.9998414962751625
Unnamed: 22 - 1.0
Unnamed: 23 - 1.0


In [38]:
# procesar Date para extraer el mes
pd.to_datetime(df.Date)

ParserError: Unknown string format: reported 30-apr-2018

In [39]:
# hay strings que no cumplen un formato estándar
df.Date[df.Date.str.len() != 11]

14                   may 2018
17       reported 30-apr-2018
31       reported 10-apr-2018
37                 9-mar-2018
38                 9-mar-2018
                 ...         
25718                     NaN
25719                     NaN
25720                     NaN
25721                     NaN
25722                     NaN
Name: Date, Length: 20836, dtype: object

In [40]:
# extraemos los meses
months = df.Date.str.findall(r'jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec')
months = months[months.notna()]
df['month'] = months.apply(lambda x: x[0] if len(x) >= 1 else np.nan)

In [41]:
# Variable de mantenimiendo de archivo
df[df["Unnamed: 23"].notna()]["Unnamed: 23"]

4415             teramo
5840    change filename
Name: Unnamed: 23, dtype: object

In [42]:
# Variable de mantenimiendo de archivo
df[df["Unnamed: 22"].notna()]["Unnamed: 22"]

1478    stopped here
Name: Unnamed: 22, dtype: object

In [43]:
# A priori area pareceria no tener importancia, pero solo tiene 801 valores únicos
# y Florida está sobrerrepresentada.
# puede ser interesante tenerlo en cuenta (aunque puede haber sesgo de más registros en esa zona)
df.Area.value_counts()

florida                 1038
new south wales          486
queensland               312
hawaii                   298
california               290
                        ... 
taipei hsien               1
misamis oriental           1
jakarta harbour            1
teramo                     1
antofagasta province       1
Name: Area, Length: 797, dtype: int64

In [44]:
# Podemos tomar las actividades más populares y agrupar el resto
# Cuantas tienen más de 50 casos?
df.Activity.value_counts()[df.Activity.value_counts() > 50]

surfing          976
swimming         916
fishing          444
spearfishing     340
bathing          165
wading           149
diving           130
standing         101
scuba diving      91
snorkeling        89
body boarding     65
Name: Activity, dtype: int64

In [45]:
# las descripciones de las heridas son demasiado específicas salvo para lo que ya está codificado en Fatal
df.Injury.value_counts()

fatal                                              815
survived                                            97
no injury                                           90
foot bitten                                         87
leg bitten                                          75
                                                  ... 
human remains washed ahore                           1
left forearm bitten, surgically amputated?           1
shark jumped into cockpit                            1
no injury to occupants; shark struck their boat      1
legs bitten fatal                                    1
Name: Injury, Length: 3628, dtype: int64

In [46]:
# La hora del día está codificada de formas irregulares y hay 354 valores distintos
df.Time.value_counts()

afternoon        187
11h00            128
morning          122
12h00            109
15h00            108
                ... 
14h37              1
06h47              1
"after lunch"      1
12h39              1
12h55              1
Name: Time, Length: 354, dtype: int64

In [47]:
# Las especies no tienen un formato consistente
df.Species.value_counts()

white shark                                                 165
shark involvement prior to death was not confirmed          105
invalid                                                     102
shark involvement not confirmed                              90
tiger shark                                                  74
                                                           ... 
2 m to 2.5 m [6.75' to 8.25'] sixgill or sevengill shark      1
lemon shark, 2.5m to 3m                                       1
3 bull sharks                                                 1
8' bull shark or caribbean reef shark                         1
hammerhead shark, 500-llb                                     1
Name: Species, Length: 1421, dtype: int64

In [48]:
# Case number parece ser la fecha, ya codificada en Date, y no aporta información
# El año no nos interesa
# Los nombres no aportan información, ni los de los investigadores, ni la edad
# los enlaces a la descripción de los casos, y demás, tampoco
df.drop(columns = df.columns[15:-1], inplace = True)
df.drop(columns = ["Case Number", "Name", "Age", "Date", "Year", "Location", "Injury", "Species", "Time"], inplace = True)
df.head()

,Type,Country,Area,Activity,Sex,Fatal (Y/N),month
0,boating,usa,california,paddling,f,n,jun
1,unprovoked,usa,georgia,standing,f,n,jun
2,invalid,usa,hawaii,surfing,m,n,jun
3,unprovoked,australia,new south wales,surfing,m,n,jun
4,provoked,mexico,colima,free diving,m,n,jun


In [49]:
# missing
100*df.isna().sum(axis = 0)/df.shape[0]

Type           75.516075
Country        75.694903
Area           77.269370
Activity       77.615364
Sex            77.697003
Fatal (Y/N)    77.595926
month          77.518174
dtype: float64

In [50]:
df.dropna(inplace = True)

In [51]:
df.Country.value_counts()[:30]

usa                 1802
australia            983
south africa         432
new zealand           92
brazil                79
papua new guinea      73
bahamas               72
mexico                63
reunion               40
italy                 35
fiji                  34
mozambique            32
philippines           31
cuba                  26
new caledonia         26
egypt                 24
spain                 24
french polynesia      22
panama                22
india                 19
croatia               18
solomon islands       14
hong kong             14
japan                 14
england               14
indonesia             13
tonga                 12
costa rica            12
jamaica               11
pacific ocean          9
Name: Country, dtype: int64

In [52]:
# Codificamos los 30 países más comunes por hemisferio (norte/sur)
# el resto los codificamos como otro
hemisferio = {
    "usa" : "norte",
    "australia" : "sur",
    "south africa" : "sur",
    "papua new guinea" : "sur",
    "new zealand" : "sur",
    "brazil" : "sur",
    "bahamas" : "norte",
    "mexico" : "norte",
    "italy" : "norte",
    "reunion" : "sur",
    "fiji" : "sur",
    "mozambique" : "sur",
    "philippines" : "sur",
    "cuba" : "norte",
    "new caledonia" : "sur",
    "spain" : "norte",
    "panama" : "norte",
    "egypt" : "norte",
    "india" : "sur",
    "french polynesia" : "sur",
    "croatia" : "norte",
    "indonesia" : "sur",
    "solomon islands" : "sur",
    "japan" : "norte",
    "greece" : "norte",
    "tonga" : "sur",
    "hong kong" : "norte",
    "england" : "norte",
    "iran" : "norte",
    "jamaica" : "norte"
}

def func_hemisferio(pais):
  if pais in hemisferio.keys():
    return hemisferio[pais]
  else:
    return 'otro'

df['hemisferio'] = df.Country.apply(func_hemisferio)

In [53]:
df['hemisferio'].value_counts()

norte    2156
sur      1902
otro      299
Name: hemisferio, dtype: int64

In [54]:
# Agrupemos el resto de valores
df.loc[df["Fatal (Y/N)"].apply(lambda x: x not in ['y', 'n']),'Fatal (Y/N)'] = 'unknown'
df.loc[df.Sex.apply(lambda x: x not in ['m', 'f']),'Sex'] = 'unknown'

In [55]:
other_activities = df.Activity.value_counts()[df.Activity.value_counts() < 50].index
df.Activity = df.Activity.apply(lambda x : 'other' if x in other_activities else x)

In [56]:
areas_principales = df.Area.value_counts()[:15].index
df.Area = df.Area.apply(lambda x: 'other' if x not in areas_principales else x)

In [57]:
df.to_csv("sharks_limpio.csv")